In [78]:
using DelimitedFiles
#using PyPlot

## Day 1

In [20]:
input = readdlm("inputs/day1.txt", ',', Int)
mapreduce(x -> floor(Int, x / 3) - 2, +, input)

3212842

In [26]:
function total_fuel(mass)
    tf = 0
    f(x) = floor(Int, x / 3) - 2
    m = mass
    while f(m) > 0
        ff = f(m)
        tf += ff
        m = ff
    end
    return tf
end
mapreduce(total_fuel, +, input)

4816402

## Day 2

In [65]:
function computer(program)
    pc = 1
    while program[pc] != 99
        opcode, x, y, r = program[pc:pc+3]
        if opcode == 1
            program[r+1] = program[x+1]+program[y+1]
        elseif opcode == 2
            program[r+1] = program[x+1]*program[y+1]
        else
            @error "Something went wrong! Got code $(opcode)."
        end
        pc += 4
    end
    return program
end

computer (generic function with 1 method)

In [68]:
input = readdlm("inputs/day2.txt", ',', Int)
input[2] = 12
input[3] = 2
result = computer(input)
print("Position 0 has: $(result[1])")

Position 0 has: 12490719

In [79]:
initial_state = readdlm("inputs/day2.txt", ',', Int)

function test_comp(input, a, b)
    input[2] = a
    input[3] = b
    return computer(input)[1]
end

test_comp (generic function with 1 method)

In [82]:
for a=0:99
    for b=0:99
        if 19690720 == test_comp(copy(initial_state), a, b)
            println("Found state: $(100*a + b).")
            break
        end
    end
end

Found state: 2003.


## Day 3

In [239]:
struct Point
    x::Int
    y::Int
end

function manhattan(a::Point, b::Point)
    return abs(a.x-b.x) + abs(a.y-b.y)
end

struct Segment
    start::Point
    stop::Point
end

function sorted_x(s::Segment)
    return sort([s.start.x, s.stop.x])
end

function sorted_y(s::Segment)
    return sort([s.start.y, s.stop.y])
end

function move(p::Point, dir::Symbol, d::Int)
    if dir == :R
        stop = Point(p.x+d, p.y)
    elseif dir == :L
        stop = Point(p.x-d, p.y)
    elseif dir == :U
        stop = Point(p.x, p.y+d)
    elseif dir == :D
        stop = Point(p.x, p.y-d)
    end
    return Segment(p, stop)
end

function wire_dir(s::Segment)
    if s.start.x == s.stop.x 
        return :V
    else
        return :H
    end
end

function intersect(s1::Segment, s2::Segment)
    no_intersection = (false, Point(0, 0))
    if wire_dir(s1) == wire_dir(s2)
        return no_intersection #for now assume collinear wires cannot intersect
    else
        if wire_dir(s1) == :H
            x1, x2 = sorted_x(s1)
            y1, y2 = sorted_y(s2)
            if (x1 <= s2.start.x <= x2) && (y1 <= s1.start.y <= y2)
                return (true, Point(s2.start.x, s1.start.y))
            else
                return no_intersection
            end
        else
            x1, x2 = sorted_x(s2)
            y1, y2 = sorted_y(s1)
            if (x1 <= s1.start.x <= x2) && (y1 <= s2.start.y <= y2)
                return (true, Point(s1.start.x, s2.start.y))
            else
                return no_intersection
            end
        end
    end
end

function parse_input_moves(input)
    moves = split(input, ',')
    return [(Symbol(m[1]), parse(Int, m[2:end])) for m in moves]
end

function build_wires(moves)
    wires = Array{Segment, 1}()
    start = Point(0,0)
    for m = moves
        new_wire = move(start, m...)
        push!(wires, new_wire)
        start = new_wire.stop
    end
    return wires
end     

function plot_wires(w1, w2)
    x = [0]
    y = [0]
    for w=w1
        push!(x, w.stop.x)
        push!(y, w.stop.y)
    end
    plot(x,y,"ro-")
    x = [0]
    y = [0]
    for w=w2
        push!(x, w.stop.x)
        push!(y, w.stop.y)
    end
    plot(x,y,"bo-")
end

function find_interection_distances(wires1, wires2)
    intersections = []
    for w1 = wires1
        for w2 = wires2
            test = intersect(w1, w2)
            if test[1] 
                push!(intersections, test[2])
                #print(test[2])
            end
        end
    end
    dst(p) = manhattan(p, Point(0,0))
    return sort([dst(p) for p in intersections if dst(p)>0])
end
                
function distance_along_wire(w::Segment, p::Point)
    return manhattan(w.start, p)
end
                
function wire_length(w::Segment)
    return manhattan(w.start, w.stop)
end
                                
function find_intersection_delays(wires1, wires2)
     delays = []
     for (j, w1) = enumerate(wires1)
        for (k, w2) = enumerate(wires2)
            test = intersect(w1, w2)
            if test[1]
                if test[2] == Point(0,0)
                    continue
                end
                #println(test[2])
                prev_delay_1 = mapreduce(wire_length, +, wires1[1:j-1])
                prev_delay_2 = mapreduce(wire_length, +, wires2[1:k-1])
                #println(prev_delay_1, " ", prev_delay_2)
                d1 = distance_along_wire(w1, test[2])
                d2 = distance_along_wire(w2, test[2])
                #println(d1, " ", d2)
                push!(delays, d1+d2+prev_delay_1+prev_delay_2)
            end
        end    
    end
    return sort(delays)                
end

find_intersection_delays (generic function with 1 method)

In [240]:
L1 = parse_input_moves("R8,U5,L5,D3")
L2 = parse_input_moves("U7,R6,D4,L4")
W1 = build_wires(L1)
W2 = build_wires(L2);
find_intersection_delays(W1,W2)

2-element Array{Any,1}:
 30
 40

In [241]:
L1 = parse_input_moves("R75,D30,R83,U83,L12,D49,R71,U7,L72")
L2 = parse_input_moves("U62,R66,U55,R34,D71,R55,D58,R83")
W1 = build_wires(L1)
W2 = build_wires(L2);
find_interection_distances(W1,W2);
find_intersection_delays(W1,W2)

4-element Array{Any,1}:
 610
 624
 726
 850

In [242]:
L1 = parse_input_moves("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51")
L2 = parse_input_moves("U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")
W1 = build_wires(L1)
W2 = build_wires(L2);
find_interection_distances(W1,W2);
find_intersection_delays(W1,W2)

5-element Array{Any,1}:
 410
 516
 636
 650
 700

In [245]:
input = readlines("inputs/day3.txt")
L1 = parse_input_moves(input[1])
L2 = parse_input_moves(input[2])
W1 = build_wires(L1)
W2 = build_wires(L2);
its = find_interection_distances(W1,W2);
println("Distance to nearest intersection: $(its[1])")
dels = find_intersection_delays(W1,W2);
println("Delay to nearest intersection: $(dels[1])")

Distance to nearest intersection: 1626
Delay to nearest intersection: 27330


## Day 4

In [18]:
function is_valid(num::Int)
    d = reverse(digits(num))
    return any((d[j] == d[j+1] for j=1:length(d)-1)) && all((d[j] <= d[j+1] for j=1:length(d)-1))
end

is_valid (generic function with 1 method)

In [42]:
count((is_valid(n) for n=240920:789857))

1154

In [44]:
function has_repeats(d::Array{Int, 1})
    return any((count(x -> x==d[j], d) == 2 for j=1:length(d)-1 if d[j]==d[j+1]))
end

function is_valid2(num::Int)
    d = reverse(digits(num))
    return all((d[j] <= d[j+1] for j=1:length(d)-1)) && has_repeats(d)
end    

is_valid2 (generic function with 1 method)

In [45]:
count((is_valid2(n) for n=240920:789857))

750

## Day 5

In [94]:
rndict = Dict(1 => 3, 2 => 3, 
              3 => 1, 4 => 1, 
              5 => 2, 6 => 2, 
              7 => 3, 8 => 3,
              99 => 0)

struct OpCode
    instr::Int
    mode::Array{Int, 1}
    N::Int
end

function OpCode(c::Int)
    digs = digits(c)
    oc = digs[1] + (length(digs) > 1 ? 10*digs[2] : 0)
    N = rndict[oc]
    mode = zeros(Int, N)
    mode[1:length(digs[3:end])] = digs[3:end]
    return OpCode(oc, mode, N)
end 


function computer2(program, input; pc=1)
    #pc = 1
    jump_flag = false
    output = []
    input_idx = 1
    
    function load(dst)
        program[dst] = input[input_idx]
        input_idx += 1
    end
    
    function store(data)
        push!(output, data)
    end    
    
    function jtrue(x, y)
        if x != 0 
            pc = y+1
            jump_flag = true
        end
    end
    
    function jfalse(x, y)
        if x == 0  
            pc = y+1 
            jump_flag = true
        end
    end
    
    lt(x, y) = Int(x < y)
    
    eq(x, y) = Int(x == y)
        
    
    while program[pc] != 99 && pc < length(program)
        jump_flag = false
        oc = OpCode(program[pc])     
        println(oc)

        if oc.N == 3 || oc.N == 2
                    
            x, y = program[pc+1:pc+2]
            
            a = oc.mode[1] == 0 ? program[x+1] : x
            b = oc.mode[2] == 0 ? program[y+1] : y
            
            println("A = $(a), B = $(b)")
                        
            if oc.N == 3
                r = program[pc+3]
                if oc.mode[3] == 1
                    error("Invalide modesetting for R register!")
                end
            println("R = $(r)")
            end
            
            if oc.instr == 1
                program[r+1] = a+b
            elseif oc.instr == 2
                program[r+1] = a*b
            elseif oc.instr == 5
                jtrue(a, b)
            elseif oc.instr == 6
                jfalse(a, b)
            elseif oc.instr == 7
                program[r+1] = lt(a,b)
            elseif oc.instr == 8
                program[r+1] = eq(a,b)
            end
            
            
        elseif oc.N == 1
            x = program[pc+1]
            println("X = $(x)")
            if oc.instr == 3
                load(x+1)
                println("loaded to $(x+1)")
            elseif oc.instr == 4
                if oc.mode[1] == 1
                    store(x)
                else
                    store(program[x+1])
                end
            end
        else
            @error "Something went wrong! Got code $(oc.instr)."
        end
        pc += (jump_flag ? 0 : oc.N+1)
        println("PC = $(pc)")
    end
    return program, output, pc
end

computer2 (generic function with 1 method)

In [122]:
input = readdlm("inputs/day5.txt", ',', Int)[:];

In [123]:
computer2([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99], [9])

([3, 21, 1008, 21, 8, 20, 1005, 20, 22, 107  …  1000, 1, 20, 4, 20, 1105, 1, 46, 98, 99], Any[1001])

In [124]:
computer2(input, [5])

([314, 225, 1, 225, 6, 6, 1105, 1, 238, 225  …  224, 674, 1001, 223, 1, 223, 4, 223, 99, 226], Any[3419022])

## Day 6

In [1]:
mutable struct CB
    name::String
    orbiters::Array{Union{CB,Nothing}, 1}
    orbit_order::Int
    parent::Union{CB, Nothing}
end

In [44]:
function parse_orbit_map(map)
    bodies = Dict()
    for orbit=map
        a, b = split(orbit, ')')
        if b in keys(bodies)
            bobj = bodies[b]
        else
            bobj = CB(b, [], -1, nothing)
            bodies[b] = bobj
        end
        if a in keys(bodies)
            aobj = bodies[a]
            push!(aobj.orbiters, bobj)
        else
            aobj = CB(a, [bobj], -1, nothing)
            bodies[a] = aobj
        end
        bobj.parent = aobj
    end
    
    return bodies
    
end

function count_orbits(bodies, head_name="COM")
    
    function add_orbit(object)
        object.orbit_order += 1
        for o=object.orbiters
            add_orbit(o)
        end
    end
    
    for obj=values(bodies)
        add_orbit(obj)
    end
    
    total = 0
    for obj=values(bodies)
        total += obj.orbit_order
    end
    
    return total
end

function get_edges(object)
    edges = isa(object.parent, CB) ? [object.parent.name] : []
    for obj in object.orbiters
        push!(edges, obj.name)
    end
    return edges
end

function find_distance(bodies, start, stop)
    distances = Dict()
    visited = Dict()
    for k = keys(bodies)
        distances[k] = 0
        visited[k] = false
    end
    
    nodes = []
    
    push!(nodes, start)
    visited[start] = true
    
    while length(nodes) > 0
        n = pop!(nodes)
        for edge = get_edges(bodies[n])
            if visited[edge]
                continue
            end
            distances[edge] = distances[n] + 1
            push!(nodes, edge)
            visited[edge] = true
        end 
    end
    
    return distances[stop]
end
    
    
    
    
    
    

find_distance (generic function with 1 method)

In [5]:
test = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L"""
test_map = split(test, '\n');
bodies = parse_orbit_map(test_map)
count_orbits(bodies)

42

In [47]:
orbit_map = readlines("./inputs/day6.txt");
bodies = parse_orbit_map(orbit_map)
count_orbits(bodies)

142915

In [49]:
find_distance(bodies, "YOU", "SAN") - 2

283

In [45]:
test2 = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN"""
test_map2 = split(test2, '\n');
bodies = parse_orbit_map(test_map2)
count_orbits(bodies)

54

In [46]:
find_distance(bodies, "YOU", "SAN")

6

## Day 7

In [54]:
function amplifier_output(phases, program)
    output = 0
    for j = 1:5
        output = computer2(copy(program), [phases[j], output])[2][1]
    end
    return output
end

amplifier_output (generic function with 1 method)

In [58]:
program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
phases = [4,3,2,1,0]
@assert amplifier_output(phases, program) == 43210

In [57]:
program = [3,23,3,24,1002,24,10,24,1002,23,-1,23,
101,5,23,23,1,24,23,23,4,23,99,0,0]
phases = [0,1,2,3,4]
@assert amplifier_output(phases, program) == 54321

In [60]:
program = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
phases = [1,0,4,3,2]
@assert amplifier_output(phases, program) == 65210

In [79]:
using Combinatorics
phases = permutations([0,1,2,3,4])
program = readdlm("inputs/day7.txt", ',', Int)[:];

In [81]:
maximum([amplifier_output(p, program) for p in phases])

13848

In [87]:
function amplifier_feedback(phases, program)
    progs = [copy(program) for x = 1:5]
    pc = [1 for x=1:5]
    output = 0
    done = false
    iter = 0
    while iter < 10000
        done = all([progs[j][pc[j]] == 99 for j=1:5])
        for idx=1:5
            progs[idx], output, pc[idx] = computer2(progs[idx], [phases[idx], output], pc=pc[idx])
        end
        iter += 1
        println(iter)
    end
    return output, iter
end

amplifier_feedback (generic function with 1 method)

In [92]:
program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
phases = [9,8,7,6,5]
#amplifier_feedback(phases, program)

5-element Array{Int64,1}:
 9
 8
 7
 6
 5

In [95]:
computer2(program, [0, 9])

OpCode(3, [0], 1)
X = 26
loaded to 27
PC = 3
OpCode(1, [0, 1, 0], 3)
A = 0, B = -4
R = 26
PC = 7
OpCode(3, [0], 1)
X = 27
loaded to 28
PC = 9
OpCode(2, [0, 1, 0], 3)
A = 9, B = 2
R = 27
PC = 13
OpCode(1, [0, 0, 0], 3)
A = 18, B = -4
R = 27
PC = 17
OpCode(4, [0], 1)
X = 27
PC = 19
OpCode(1, [0, 1, 0], 3)
A = 4, B = -1
R = 28
PC = 23
OpCode(5, [0, 1], 2)
A = 3, B = 6
PC = 7
OpCode(3, [0], 1)
X = 27


BoundsError: BoundsError: attempt to access 2-element Array{Int64,1} at index [3]